In [22]:
import pandas as pd
from sklearn.cluster import KMeans
import json
user_info = json.load(open("aus_user_feature.json"))

In [23]:
user_info = dict(user_info)
# extract the data, extract the value of the dictionary
user_info_value = [user_info[k] for k in user_info.keys()]
kmeans = KMeans(n_clusters=16, random_state=42, init="k-means++",max_iter=300).fit(user_info_value)

In [24]:
user_label = {list(user_info.keys())[i]:kmeans.labels_[i] for i in range(len(kmeans.labels_))}
user_label_df = pd.DataFrame(user_label.items(), columns=["user", "label"])
label_user_count = user_label_df.groupby("label").size()

In [27]:
user_label_df["label"] = user_label_df["label"].astype(str)

In [28]:
new_clean_user_item = pd.read_csv("aus_user_item.csv")
new_clean_user_item["steam_id"] = new_clean_user_item["steam_id"].astype(str)

In [29]:
user_label_df = user_label_df.rename({"user":"steam_id"},axis = 1)
user_label_df["steam_id"] = user_label_df["steam_id"].astype(str)

In [30]:
# Merge 2 tables, different joins.
new_clean_user_item_with_label = pd.merge(
     new_clean_user_item, user_label_df, how="left", on=["steam_id"]
)
new_clean_user_item_with_label.dropna(inplace=True)

In [43]:
item_label = new_clean_user_item_with_label.loc[:,["item_id", "label"]]

In [32]:
item_coverage_series = item_label.groupby("item_id")['label'].nunique()
item_coverage_series=item_coverage_series.reset_index(name='label_count')
item_coverage_series["width_coverage"] = item_coverage_series["label_count"] / 16


,item_id,label_count,width_coverage
0,10,10,0.6250
1,20,9,0.5625
2,30,9,0.5625
3,40,9,0.5625
4,50,8,0.5000
...,...,...,...
10973,528580,2,0.1250
10974,528660,1,0.0625
10975,529670,2,0.1250
10976,529820,1,0.0625


In [44]:
# deep coverage
item_max_group = item_label.groupby('item_id')['label'].apply(lambda x: x.value_counts().max()).reset_index(name='max_count')
item_max_group

,item_id,max_count
0,10,5614
1,20,4254
2,30,2172
3,40,2087
4,50,4212
...,...,...
10973,528580,5
10974,528660,2
10975,529670,3
10976,529820,1


In [36]:
item_user_count = new_clean_user_item_with_label.groupby("item_id")['steam_id'].nunique()
item_user_count = item_user_count.reset_index(name='user_count')

In [37]:
item_depth = item_max_group.join(item_user_count.set_index('item_id')['user_count'], on='item_id')
item_depth["depth_coverage"] = item_depth["max_count"] / item_depth["user_count"]
item_depth # item覆盖用户深度 用max_group中买了该item的人数 / 所有买了该item的人数

,item_id,max_count,user_count,depth_coverage
0,10,5614,9611,0.584122
1,20,4254,6268,0.678685
2,30,2172,3431,0.633052
3,40,2087,3242,0.643738
4,50,4212,6216,0.677606
...,...,...,...,...
10973,528580,5,6,0.833333
10974,528660,2,2,1.000000
10975,529670,3,4,0.750000
10976,529820,1,1,1.000000


In [38]:
# merge the 2 series together
item_features = pd.concat([item_coverage_series, item_depth],axis=1)
item_features.reset_index(inplace=True)

In [41]:
item_features_csv = item_features[["item_id", "width_coverage", "depth_coverage"]]

In [42]:
item_features_csv.to_csv("features_dataset/item_features_with_depth.csv")